## Creacion de DataFrames auxiliares

En este notebook vamos a crear los data frames auxiliares que tendremos que usar para las funciones que vamos a desarrollar 

### Cargamos las librerias necesarias

In [25]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import warnings
warnings.filterwarnings("ignore")

### Cargamos los CSV necesarios para extraer la informacion necesaria

In [26]:
df_reviews = pd.read_csv("data2/user_reviews_limpio7.csv", encoding="utf-8")
df_output = pd.read_csv("data2/output_steam_games_limpio8.csv", encoding="utf-8")
df_items = pd.read_csv("data2/user_items_limpio5.csv", encoding="utf-8")

In [4]:
df_items.columns

Index(['user_id', 'items_count', 'item_id', 'item_name', 'playtime_forever'], dtype='object')

In [5]:
df_reviews.columns

Index(['user_id', 'item_id', 'recommend', 'year_posted', 'sentiment_analysis'], dtype='object')

In [6]:
df_output.columns

Index(['app_name', 'id', 'developer', 'price_number', 'release_year',
       'main_genre'],
      dtype='object')

In [32]:
df_items["playtime_hour"]= df_items["playtime_forever"]/60
df_items

,user_id,items_count,item_id,item_name,playtime_forever,playtime_hour
0,76561197970982479,277,20,Team Fortress Classic,0.0,0.000000
1,76561197970982479,277,30,Day of Defeat,7.0,0.116667
2,76561197970982479,277,40,Deathmatch Classic,0.0,0.000000
3,76561197970982479,277,60,Ricochet,0.0,0.000000
4,76561197970982479,277,3830,Psychonauts,333.0,5.550000
...,...,...,...,...,...,...
3002796,76561198329548331,7,304930,Unturned,677.0,11.283333
3002797,76561198329548331,7,346330,BrainBread 2,0.0,0.000000
3002798,76561198329548331,7,373330,All Is Dust,0.0,0.000000
3002799,76561198329548331,7,521570,You Have 10 Seconds 2,4.0,0.066667


### Funcion PlayTimeGenre
En este caso nos piden una funcion que devuelva el año con mas horas jugadas para un genero especificado, por lo que seran necesarias las columnas main_genre, id y Release_year de df_output y la columna playtime_forever e item_id de df_items

In [13]:
df_output2 = df_output[["main_genre", "release_year","id"]]
df_items2 = df_items[["playtime_hour", "item_id"]]
df_PlayTimeGenre_merge = pd.merge(df_items2, df_output2, left_on='item_id', right_on="id")
df_PlayTimeGenre_hour = df_PlayTimeGenre_merge.groupby(["main_genre", "release_year"])["playtime_hour"].sum().reset_index()
df_PlayTimeGenre_hour

,main_genre,release_year,playtime_hour
0,Action,1984,6.400000
1,Action,1990,313.116667
2,Action,1991,19.116667
3,Action,1992,12.683333
4,Action,1993,33.700000
...,...,...,...
232,Utilities,2016,1305.183333
233,Video Production,2015,0.000000
234,Web Publishing,2005,1564.116667
235,Web Publishing,2015,3.100000


In [12]:
df_output2_ = df_output[["main_genre", "release_year","id"]]
df_items2_ = df_items[["playtime_forever", "item_id"]]
df_PlayTimeGenre_merge_ = pd.merge(df_items2_, df_output2_, left_on='item_id', right_on="id")
df_PlayTimeGenre_forever = df_PlayTimeGenre_merge_.groupby(["main_genre", "release_year"])["playtime_forever"].sum().reset_index()
df_PlayTimeGenre_forever

,main_genre,release_year,playtime_forever
0,Action,1984,384.0
1,Action,1990,18787.0
2,Action,1991,1147.0
3,Action,1992,761.0
4,Action,1993,2022.0
...,...,...,...
232,Utilities,2016,78311.0
233,Video Production,2015,0.0
234,Web Publishing,2005,93847.0
235,Web Publishing,2015,186.0


In [14]:
archivo_limpio = "data2/df_PlayTimeGenre_hour.csv"
df_PlayTimeGenre_hour.to_csv(archivo_limpio, index=False, encoding="utf-8")
print(f"Se guardó el archivo {archivo_limpio}") 

Se guardó el archivo data2/df_PlayTimeGenre_hour.csv


In [15]:
archivo_limpio2 = "Datasets/df_PlayTimeGenre_hour.parquet"
df_PlayTimeGenre_hour.to_parquet(archivo_limpio2, index=False)
print(f"Se guardó el archivo {archivo_limpio2}")

Se guardó el archivo Datasets/df_PlayTimeGenre_hour.parquet


In [16]:
archivo_limpio3 = "data2/df_PlayTimeGenre_forever.csv"
df_PlayTimeGenre_forever.to_csv(archivo_limpio3, index=False, encoding="utf-8")
print(f"Se guardó el archivo {archivo_limpio3}") 

Se guardó el archivo data2/df_PlayTimeGenre_forever.csv


In [17]:
archivo_limpio4 = "Datasets/df_PlayTimeGenre_forever.parquet"
df_PlayTimeGenre_forever.to_parquet(archivo_limpio4, index=False)
print(f"Se guardó el archivo {archivo_limpio4}")

Se guardó el archivo Datasets/df_PlayTimeGenre_forever.parquet


### Funcion UserForGenre
En este caso nos piden una funcion que devuelva el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año, por lo que seran necesarias las columnas  Genre y Release_year de df_output y las columnas playtime_forever y user_id de df_items

In [36]:
df_items_filtrado = df_items[["user_id","item_id","playtime_hour"]]
df_reviews_filtrado = df_reviews[["user_id","item_id","year_posted"]]
df_output_filtrado = df_output[["id","main_genre"]]

df_merged = pd.merge(df_items_filtrado, df_reviews_filtrado, on="item_id", validate="one_to_one")
df_merged = pd.merge(df_merged, df_output_filtrado, left_on="item_id", right_on="id")

df_user_genre_hours = df_merged.groupby(['user_id', 'main_genre', 'year_posted'])['playtime_hour'].sum().reset_index()
max_hours_idx = df_user_genre_hours.groupby(['main_genre', 'year_posted'])['playtime_hour'].idxmax()
df_result = df_user_genre_hours.loc[max_hours_idx][['user_id', 'main_genre', 'year_posted', 'playtime_hour']]
df_result

MergeError: Merge keys are not unique in either left or right dataset; not a one-to-one merge

In [ ]:
'''
archivo_limpio3 = "data2/df_UserForGenre.csv"
df_UserForGenre.to_csv(archivo_limpio3, index=False, encoding="utf-8")
print(f"Se guardó el archivo {archivo_limpio3}")
'''

NameError: name 'df_UserForGenre' is not defined

In [ ]:
'''
archivo_limpio4 = "Datasets/df_UserForGenre.parquet"
df_UserForGenre.to_parquet(archivo_limpio4, index=False)
print(f"Se guardó el archivo {archivo_limpio4}")
'''

Se guardó el archivo Datasets/df_UserForGenre.parquet


### Funcion UsersRecommend
En este caso nos piden una funcion que devuelve el top 3 de juegos MÁS recomendados por usuarios para el año dado, por lo que seran necesarias la columna id y app_name de df_output y las columnas item_id, recommend y year_posted de df_reviews

In [22]:
df_output3_ = df_output[["id", "app_name"]].drop_duplicates()
df_reviews2_ = df_reviews[["item_id", "recommend", "year_posted","sentiment_analysis"]]
df_reviews_merged_ = pd.merge(df_reviews2_, df_output3_, left_on="item_id", right_on="id")
positive_reviews_ = df_reviews_merged_[(df_reviews_merged_["recommend"]== True) & (df_reviews_merged_["sentiment_analysis"] >=1)]
grouped_reviews_ = positive_reviews_.groupby(["year_posted", "item_id", "app_name"]).size().reset_index(name="recommendation_count")
df_UsersRecommend_ = (
    grouped_reviews_.sort_values(by=["year_posted", "recommendation_count"], ascending=[True, False])
                   .groupby("year_posted")
                   .head(3)
                   .reset_index(drop=True)
)
df_UsersRecommend_

,year_posted,item_id,app_name,recommendation_count
0,2010,440,Team Fortress 2,10
1,2010,1250,Killing Floor,6
2,2010,630,Alien Swarm,4
3,2011,440,Team Fortress 2,72
4,2011,620,Portal 2,24
5,2011,105600,Terraria,20
6,2012,440,Team Fortress 2,234
7,2012,4000,Garry's Mod,36
8,2012,105600,Terraria,36
9,2013,440,Team Fortress 2,938


In [23]:
archivo_limpio5 = "data2/df_UsersRecommend2.csv"
df_UsersRecommend_.to_csv(archivo_limpio5, index=False, encoding="utf-8")
print(f"Se guardó el archivo {archivo_limpio5}") 

Se guardó el archivo data2/df_UsersRecommend2.csv


In [24]:
archivo_limpio6 = "Datasets/df_UsersRecommend2.parquet"
df_UsersRecommend_.to_parquet(archivo_limpio6, index=False)
print(f"Se guardó el archivo {archivo_limpio6}")

Se guardó el archivo Datasets/df_UsersRecommend2.parquet


### Funcion UsersWorstDeveloper
En este caso nos piden una funcion que devuelve el top 3 de desarrolladoras con juegos MENOS recomendados por usuarios para el año dado. (reviews.recommend = False y comentarios negativos)

In [28]:
df_output4 = df_output[["id", "developer"]].drop_duplicates()
df_reviews4 = df_reviews[["item_id", "recommend", "year_posted","sentiment_analysis"]]
df_reviews_merged4 = pd.merge(df_reviews4, df_output4, left_on="item_id", right_on="id")
negative_reviews4 = df_reviews_merged4[(df_reviews_merged4["recommend"]== False) & (df_reviews_merged4["sentiment_analysis"] == 0)]
grouped_reviews4 = negative_reviews4.groupby(["year_posted", "item_id", "developer"]).size().reset_index(name="recommendation_count")
df_UsersWorstDeveloper = (
    grouped_reviews4.sort_values(by=["year_posted", "recommendation_count"], ascending=[True, False])
                   .groupby("year_posted")
                   .head(3)
                   .reset_index(drop=True)
)
df_UsersWorstDeveloper

,year_posted,item_id,developer,recommendation_count
0,2011,440,Valve,1
1,2011,18700,Broken Rules,1
2,2011,33460,Ubisoft Montpellier,1
3,2012,440,Valve,1
4,2012,42920,NeoCoreGames,1
5,2012,55110,Volition,1
6,2013,275850,Hello Games,52
7,2013,730,Valve,32
8,2013,440,Valve,25
9,2014,221100,Bohemia Interactive,19


In [29]:
archivo_limpio7 = "data2/df_UsersWorstDeveloper.csv"
df_UsersWorstDeveloper.to_csv(archivo_limpio7, index=False, encoding="utf-8")
print(f"Se guardó el archivo {archivo_limpio7}") 

Se guardó el archivo data2/df_UsersWorstDeveloper.csv


In [30]:
archivo_limpio8 = "Datasets/df_UserWorstDeveloper.parquet"
df_UsersWorstDeveloper.to_parquet(archivo_limpio8, index=False)
print(f"Se guardó el archivo {archivo_limpio8}")

Se guardó el archivo Datasets/df_UserWorstDeveloper.parquet


### Funcion Sentiment_analysis
En este caso nos piden una funcion que devuelve un diccionario con el nombre de la desarrolladora como llave y una lista con la cantidad total de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento como valor.

In [40]:
df_reviews0 = df_reviews[["user_id", "item_id","recommend","year_posted","sentiment_analysis"]]
df_reviews1 = pd.merge(df_reviews0, df_output[["id","app_name","developer"]], left_on="item_id", right_on="id")
df_reviews02 = df_reviews1[["user_id","app_name","developer","recommend","year_posted","sentiment_analysis"]]
df_Sentiment_analysis = df_reviews02.groupby(["developer","sentiment_analysis"]).size().reset_index(name="sentiment_analysis_count")
df_Sentiment_analysis

,developer,sentiment_analysis,sentiment_analysis_count
0,07th Expansion,0,1
1,07th Expansion,1,2
2,"10th Art Studio,Adventure Productions",0,1
3,"10th Art Studio,Adventure Productions",1,1
4,10tons Ltd,1,1
...,...,...,...
3960,"インレ,Inre",1,4
3961,橘子班,0,1
3962,橘子班,1,2
3963,橘子班,2,1


In [41]:
archivo_limpio10 = "data2/df_Sentiment_analysis.csv"
df_Sentiment_analysis.to_csv(archivo_limpio10, index=False, encoding="utf-8")
print(f"Se guardó el archivo {archivo_limpio10}") 

Se guardó el archivo data2/df_Sentiment_analysis.csv


In [42]:
archivo_limpio11 = "Datasets/df_Sentiment_analysis.parquet"
df_Sentiment_analysis.to_parquet(archivo_limpio11, index=False)
print(f"Se guardó el archivo {archivo_limpio11}")

Se guardó el archivo Datasets/df_Sentiment_analysis.parquet
